In [9]:
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta, time
import calendar
from mpl_toolkits.mplot3d import Axes3D

In [96]:
# Take in data
# data_df = pd.read_csv('La_Ola_data_new.csv')
data_df = pd.read_csv('20030625.csv')

zenith = data_df[data_df.columns[3]] # zenith angle column
irr =  data_df[data_df.columns[2]]   # solar irradiance column
time = data_df[data_df.columns[1]]   # time column
date = data_df[data_df.columns[0]]   # date column


def extract_day_n(data_df, n):
    unique_dates = data_df['DATE (MM/DD/YYYY)'].unique()
    return data_df[data_df['DATE (MM/DD/YYYY)'] == unique_dates[n]]

def extract_zen_under_n(data_df, n):
    cond_bool = data_df.apply(lambda x: x['Zenith Angle [degrees]'] < n, axis=1)
    return data_df[cond_bool]

def extract_zen_over_n(data_df, n):
    cond_bool = data_df.apply(lambda x: x['Zenith Angle [degrees]'] > n, axis=1)
    return data_df[cond_bool]

data_df['Zenith Angle [radians]'] = data_df.apply(lambda x: np.deg2rad(x['Zenith Angle [degrees]']), axis=1)

In [104]:
print(data_df.head)

<bound method NDFrame.head of        DATE (MM/DD/YYYY)    CST  Global PSP-A [W/m^2]  Global PSP-B [W/m^2]  \
0              6/25/2003  05:15              -2.52450              -2.00050   
1              6/25/2003  05:16              -2.08760              -1.47810   
2              6/25/2003  05:17              -1.48420              -0.99684   
3              6/25/2003  05:18              -1.00560              -0.37811   
4              6/25/2003  05:19              -0.51322               0.10312   
5              6/25/2003  05:20               0.02774               0.63935   
6              6/25/2003  05:21               0.76983               1.33370   
7              6/25/2003  05:22               1.58130               2.09680   
8              6/25/2003  05:23               2.33720               2.80490   
9              6/25/2003  05:24               3.16950               3.63670   
10             6/25/2003  05:25               3.85610               4.35170   
11             6/25/20

In [98]:
# Create a day label (0-366, possibly 366th date because of leap year)
date1 = np.linspace(0, 365, num = 366, endpoint = True, retstep = False, dtype = None)

In [99]:
# convert time to minutes
time = np.asarray(time)
time = np.unique(time)

time1 = datetime.strptime(time[0], "%H:%M")
time1 = time1.minute + time1.hour * 60 #convert time to minutes
time1 = np.asmatrix(time1)

for j in range(1,time.size):
    timei = datetime.strptime(time[j], "%H:%M")
    timei = timei.minute + timei.hour * 60 #convert time to minutes
    timei = np.asmatrix(timei)
    time1 = np.hstack((time1, timei))

In [122]:
# Cut time so that the time is only daylight hours.
time1 = np.asarray(time1)
time1 = np.unique(time1)
start = 540
end = 1021

# Filter down the time between the minute range of 540 and 1020.
for j in range(0,time1.size):
    if time1[j] == '540':
        start = j
    if time1[j] == '1020':
        end = j+1
        
time1 = time1
time1 = np.asarray(time1)

time1 = list(range(0,8*60 + 1))

In [125]:
# Create a 2D matrix of the X and Y coordinates in preparation for making XYZ tuples for coordinates
# for the plotting function.

# Here, grid1 is the X = time1 grid.
# and,  grid2 is the Y = date1 grid.
grid1, grid2 = np.meshgrid(time1, date1, sparse=False, indexing='ij')

row, col = grid2.shape
row = row
col = col
print(row, col)

481 366


In [126]:
# Normalize irradiance with Zenith Angle
data_df['Normalized Irradiance'] = data_df.apply(
    lambda x: x['Global PSP-A [W/m^2]'] / np.abs(np.cos(x['Zenith Angle [radians]'])),
    axis = 1
)



In [118]:
print(norm_irr)
print(norm_irr.shape)

[[  -6.05974872  -22.55313373   -2.73354148 ...,  231.66373897
    98.91997348  119.66114557]]
(1, 266787)


In [131]:
data_df[data_df['Global PSP-A [W/m^2]'] < -10]

,DATE (MM/DD/YYYY),CST,Global PSP-A [W/m^2],Global PSP-B [W/m^2],Zenith Angle [degrees],Zenith Angle [radians],Normalized Irradiance
679,6/25/2003,16:34,-11.283,-10.5590,53.03159,0.925576,-18.762016
680,6/25/2003,16:35,-12.386,-11.5080,53.23170,0.929068,-20.692288
681,6/25/2003,16:36,-13.683,-12.8000,53.43189,0.932562,-22.966610
682,6/25/2003,16:37,-13.725,-12.6970,53.63193,0.936054,-23.146187
683,6/25/2003,16:38,-11.956,-10.8340,53.83203,0.939546,-20.259103
687,6/25/2003,16:42,-11.186,-10.0570,54.63188,0.953506,-19.325273
688,6/25/2003,16:43,-10.666,-9.5279,54.83170,0.956994,-18.518000
13087,7/9/2003,19:39,-10.070,-9.8374,88.39763,1.542830,-360.118897
13088,7/9/2003,19:40,-10.223,-10.0510,88.56030,1.545669,-406.887821
13089,7/9/2003,19:41,-10.389,-10.1060,88.72169,1.548486,-465.689261


In [127]:
# Shape irradiance into date and time size matrix

norm_irr_array = data_df['Normalized Irradiance']

irr1 = np.reshape(norm_irr_array[0:row*col], (row, col))

print(irr1.shape)
print(norm_irr_array.shape)

(481, 366)
(266787,)


In [128]:
#2D plot: time and irradiance
x = list(range(0, 266787))
y = norm_irr_array
plt.plot(x, norm_irr_array)
plt.show()

In [ ]:
#Regular 3D plot
#shape irradiance into date and time size matrix
print(col)

fig = plt.figure()
ax = fig.gca(projection='3d')

date_grid1 = grid1
time_grid1 = grid2

surf = ax.plot_surface(date_grid1, time_grid1, irr1, rstride = 15, cstride = 15, cmap = plt.cm.cool, linewidth=.5, antialiased=False, closed = True)
ax.set_xlabel('Date (Days)')
ax.set_ylabel('Time (Minutes)')
ax.set_zlabel('Solar Irradiance (W/m^2)')
plt.show()

In [ ]:
#Normalized 3D plot
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(date_grid, time_grid, irr1, rstride = 30, cstride = 30, cmap = plt.cm.coolwarm,linewidth=.5, antialiased=False)
ax.set_xlabel('Date (Days)')
ax.set_ylabel('Time (Minutes)')
ax.set_zlabel('Normalized Solar Irradiance (W/m^2)')
plt.show()

In [ ]:
#Regular 3D plot test
fig = plt.figure()
ax = fig.gca(projection='3d')
date_grid = [1, 2, 3, 4]
time_grid = [1, 2, 3, 4]
date_grid,time_grid = np.meshgrid(date_grid,time_grid)
irr1 = [1, 2, 3, 4]
surf = ax.plot_surface(date_grid, time_grid, irr1, rstride = 1, cstride = 1, cmap = plt.cm.cool, linewidth = 1, antialiased = True)
plt.show()

In [ ]:
#check normalized solar irradiance
norm = np.cos(zenith[540])

norm_irr = irr[540]/abs(norm)
print(norm)
print(irr[540])
print(norm_irr)

norm = np.cos(zenith[550])

norm_irr = irr[550]/abs(norm)
print(norm)
print(irr[550])
print(norm_irr)